In [1]:
%matplotlib inline

import itertools
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns

# Statement of Problem

You are the CEO of a space transport company in the year 2080, and your chief scientist comes in to tell you that one of your space probes has detected an alien artifact at the Jupiter Solar Lagrangian (L2) point.

You want to be the first to get to it! But you know that the story will leak soon and you only have a short time to make critical decisions. With standard technology available to anyone with a few billion dollars, a manned rocket can be quickly assembled and arrive at the artifact in 1,600 days. But with some nonstandard items you can reduce that time and beat the competition. Your accountants tell you that they can get you an immediate line of credit of $1 billion.

You can buy:

+ Big Russian engines. There are only three in the world and the Russians want $\$$400 million for each of them. Buying one will reduce the trip time by 200 days. Buying two will allow you to split your payload and will save another 100 days.
+ NASA ion engines. There are only eight of these $\$$140 million large-scale engines in the world. Each will consume 5,000 kilograms of xenon during the trip. There are 30,000 kg of xenon available worldwide at a price of $\$$2,000/kg, so 5,000 kg costs $\$$10 million. Bottom line: For each $\$$150 million fully fueled xenon engine you buy, you can take 50 days off of the trip.
+ Light payloads. For $\$$50 million each, you can send one of four return flight fuel tanks out ahead of the mission, using existing technology. Each time you do this, you lighten the main mission and reduce the arrival time by 25 days.

What’s your best strategy to get there first?

In [2]:
class Equipment(object):
    def __init__(self, num):
        self.num = num
        
    @property
    def cost(self):
        raise NotImplementedError()
    
    @property
    def days_gained(self):
        raise NotImplementedError()
     
    
class EquipmentError(Exception):
    pass
    
    
class RussianEngine(Equipment):
    def __init__(self, num):
        if 0 <= num <= 3:
            super().__init__(num)
        else:
            raise EquipmentError("number out of bounds")
    
    @property
    def cost(self):
        return self.num * 400
    
    @property
    def days_gained(self):
        if self.num == 0:
            return 0
        elif self.num == 1:
            return 200
        elif self.num == 2:
            return 300
        elif self.num == 3:
            return 300
     
    
class NasaEngine(Equipment):
    def __init__(self, num, fuelNum):
        self.fuelNum = fuelNum
        if (0 <= num <= 8) and (0 <= fuelNum <= 6):
            super().__init__(num)
        else:
            raise EquipmentError("number out of bounds")
    
    @property
    def cost(self):
        return self.num * 140 + self.fuelNum * 10
    
    @property
    def days_gained(self):
        return min(self.num, self.fuelNum) * 50
     
    
class LightPayload(Equipment):
    def __init__(self, num):
        if 0 <= num <= 4:
            super().__init__(num)
        else:
            raise EquipmentError("number out of bounds")
    
    @property
    def cost(self):
        return self.num * 50
    
    @property
    def days_gained(self):
        return self.num * 25
    
    
class Rocket(object):
    def __init__(self, nR, nN, nFuel, nL):
        self.nR = nR
        self.nN = nN
        self.nFuel = nFuel
        self.nL = nL
        try:
            self.equip = {
                'russia': RussianEngine(nR),
                'nasa': NasaEngine(nN, nFuel),
                'light': LightPayload(nL)
            }
        except EquipmentError:
            self.equip = None
            
    @property
    def cost(self):
        if self.equip is not None:
            return sum(_.cost for _ in self.equip.values())
        else:
            return None
        
    @property
    def days_gained(self):
        if self.equip is not None:
            return sum(_.days_gained for _ in self.equip.values())
        else:
            return None

In [3]:
d = []
for (nR, nN, nFuel, nL) in itertools.product(range(4), range(9), range(7), range(5)):
    r = Rocket(nR, nN, nFuel, nL)
    dr = Rocket(3 - nR, 8 - nN, 6 - nFuel, 4)
    d.append({
        'nR': nR,
        'nN': nN,
        'nFuel': nFuel,
        'nL': nL,
        'cost': r.cost,
        'days_gained': r.days_gained,
        'op_days_gained': dr.days_gained,
    })

df = pd.DataFrame(d)
df.head()

,cost,days_gained,nFuel,nL,nN,nR,op_days_gained
0,0,0,0,0,0,0,700
1,50,25,0,1,0,0,700
2,100,50,0,2,0,0,700
3,150,75,0,3,0,0,700
4,200,100,0,4,0,0,700


In [4]:
df.shape

(1260, 7)

In [5]:
inBudget = df[df.cost <= 1000]
inBudget.head()

,cost,days_gained,nFuel,nL,nN,nR,op_days_gained
0,0,0,0,0,0,0,700
1,50,25,0,1,0,0,700
2,100,50,0,2,0,0,700
3,150,75,0,3,0,0,700
4,200,100,0,4,0,0,700


In [6]:
inBudget.shape

(408, 7)

In [7]:
inBudget[inBudget.days_gained == inBudget.days_gained.max()]

,cost,days_gained,nFuel,nL,nN,nR,op_days_gained
438,1000,425,3,3,3,1,550


In [8]:
inBudget.sort_values(by='days_gained', ascending=False).head(10)

,cost,days_gained,nFuel,nL,nN,nR,op_days_gained
438,1000,425,3,3,3,1,550
442,960,400,4,2,3,1,500
475,1000,400,4,0,4,1,500
404,910,400,3,4,2,1,550
399,900,400,2,4,2,1,600
452,980,400,6,2,3,1,400
447,970,400,5,2,3,1,450
409,920,400,4,4,2,1,500
634,1000,400,0,4,0,2,600
437,950,400,3,2,3,1,550


In [11]:
weWin = df[(df.cost <= 1000) & (df.days_gained > df.op_days_gained)]
weWin.loc[:, 'margin'] = weWin.days_gained - weWin.op_days_gained
weWin.head()

/Users/zach.lamberty/anaconda2/envs/bullshit3/lib/python3.5/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/zach.lamberty/anaconda2/envs/bullshit3/lib/python3.5/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,cost,days_gained,nFuel,nL,nN,nR,op_days_gained,margin
658,1000,375,5,3,0,2,350,25
661,910,325,6,1,0,2,300,25
662,960,350,6,2,0,2,300,50
695,1000,350,6,0,1,2,300,50
